Current recall generates a null distribution for each given gene, while the previous recall method uses a single null distribution. Reanalyze the data with a single null distribution.

In [168]:
import pickle
import pandas as pd
import glob
import re
from scipy.stats import spearmanr
import numpy as np

In [169]:
# get results
dir_Lx = '../out/20.0518 Lx/'
Lx = 300
model_name = 'topfeat'
df_results = pd.read_csv('%s/L%sonly_reg_rf_boruta/model_results.csv' % (dir_Lx, Lx))
df_results = df_results.loc[df_results.model == model_name,:].copy()

In [170]:
# get y_actual and y_pred, list of pickle file names
dir_pkl = '../out/20.0518 Lx/L%sonly_reg_rf_boruta/model_perf' % Lx
pkl_files = glob.glob('%s/y_compr_*.pkl' % dir_pkl)

In [171]:
y_compr_list = []
gene_names = []

for pkl_file in pkl_files:
    gene = re.findall('.*/y_compr_(.*).pkl', pkl_file)[0]
    y_compr = pickle.load(open(pkl_file, 'rb'))

    y_compr_list.append(y_compr['te'])
    gene_names.append(gene)

In [172]:
cell_lines = []

#null rho distribution
y_actual = pd.DataFrame([i['y_actual'] for i in y_compr_list],
                        #columns=cell_lines,
                        index=gene_names)
y_actual = y_actual.T
y_inf = pd.DataFrame([i['y_pred'] for i in y_compr_list],
                     #columns=cell_lines,
                     index=gene_names)
y_inf = y_inf.T

rho,pval = spearmanr(y_actual, y_inf, axis=0)

rho_actual = rho[0:y_actual.shape[1], y_actual.shape[1]:]
rho_actual = rho_actual[np.eye(rho_actual.shape[0],dtype=bool)].reshape(rho_actual.shape[0])
rho_actual = pd.DataFrame(rho_actual, index=y_inf.columns, columns=['rho_actual'])
rho_null = rho[0:y_actual.shape[1], y_actual.shape[1]:]
rho_null = rho_null[~np.eye(rho_null.shape[0],dtype=bool)].reshape(rho_null.shape[0],-1) #remove diagonal
rho_null = np.squeeze(rho_null.reshape(1,np.prod(rho_null.shape))) #squeeze into 1d array

#calculate recall
def calcRecall(val):
    return np.sum(val > rho_null) / len(rho_null)

df_results['corr_test_recall_1null'] = df_results['corr_test'].apply(calcRecall)

In [173]:
df_results.to_csv('tmp/model_results_L%s_reg_rf_boruta_%s.csv' % (Lx,model_name), index=False)